In [4]:
#Tugas 1
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('EmployeeAnalysis').getOrCreate()

data = [
    ('James', 34, 'M', 3000, 1),
    ('Anna', 28, 'F', 4100, 2),
    ('Lee', 23, 'M', 2700, 1)
]
columns = ['Name', 'Age', 'Gender', 'Salary', 'DeptId']
df = spark.createDataFrame(data, schema=columns)
df.createOrReplaceTempView('employees')
spark.sql('SELECT * FROM employees').show()
spark.sql('SELECT Name, Age FROM employees WHERE Salary > 3000').show()
spark.sql('SELECT AVG(Salary) FROM employees').show()

+-----+---+------+------+------+
| Name|Age|Gender|Salary|DeptId|
+-----+---+------+------+------+
|James| 34|     M|  3000|     1|
| Anna| 28|     F|  4100|     2|
|  Lee| 23|     M|  2700|     1|
+-----+---+------+------+------+

+----+---+
|Name|Age|
+----+---+
|Anna| 28|
+----+---+

+------------------+
|       avg(Salary)|
+------------------+
|3266.6666666666665|
+------------------+



In [7]:
#Tugas 2
spark.sql('SELECT Gender, SUM(Salary) as TotalSalary, Age FROM employees GROUP BY Gender, Age ORDER BY Age').show()

# Menghitung rata-rata gaji per departemen
spark.sql('SELECT DeptId, AVG(Salary) AS AverageSalary FROM employees GROUP BY DeptId').show()

# Menghitung rata-rata gaji per gender
avg_salary_gender = spark.sql('SELECT Gender, AVG(Salary) AS AvgSalary FROM employees GROUP BY Gender')
avg_salary_gender.createOrReplaceTempView('avg_gender')

# Mencari karyawan dengan gaji di atas rata-rata per gender
spark.sql('''
    SELECT e.*
    FROM employees e
    JOIN avg_gender ag ON e.Gender = ag.Gender
    WHERE e.Salary > ag.AvgSalary
''').show()

# Membuat ranking karyawan berdasarkan gaji dalam departemen
spark.sql('''
    SELECT *,
           DENSE_RANK() OVER (PARTITION BY DeptId ORDER BY Salary DESC) AS SalaryRank
    FROM employees
''').show()


+------+-----------+---+
|Gender|TotalSalary|Age|
+------+-----------+---+
|     M|       2700| 23|
|     F|       4100| 28|
|     M|       3000| 34|
+------+-----------+---+

+------+-------------+
|DeptId|AverageSalary|
+------+-------------+
|     1|       2850.0|
|     2|       4100.0|
+------+-------------+

+-----+---+------+------+------+
| Name|Age|Gender|Salary|DeptId|
+-----+---+------+------+------+
|James| 34|     M|  3000|     1|
+-----+---+------+------+------+

+-----+---+------+------+------+----------+
| Name|Age|Gender|Salary|DeptId|SalaryRank|
+-----+---+------+------+------+----------+
|James| 34|     M|  3000|     1|         1|
|  Lee| 23|     M|  2700|     1|         2|
| Anna| 28|     F|  4100|     2|         1|
+-----+---+------+------+------+----------+



In [11]:
# Menggunakan window functions untuk menemukan top 3 karyawan dalam setiap kelompok usia
spark.sql('''
    SELECT Name, Age, Salary, 
           ROW_NUMBER() OVER (PARTITION BY Age ORDER BY Salary DESC) as rank
    FROM employees
''').createOrReplaceTempView('ranked_employees')

# Mengambil karyawan dengan rank 1 sampai 3
spark.sql('''
    SELECT Name, Age, Salary, rank
    FROM ranked_employees
    WHERE rank <= 3
''').show()


+-----+---+------+----+
| Name|Age|Salary|rank|
+-----+---+------+----+
|  Lee| 23|  2700|   1|
| Anna| 28|  4100|   1|
|James| 34|  3000|   1|
+-----+---+------+----+



In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Pertemuan9').getOrCreate()

# Data setup for complex SQL queries
employees = spark.createDataFrame([
    ('James', 34, 'M', 3000, 1),
    ('Anna', 28, 'F', 4100, 2),
    ('Lee', 23, 'M', 2700, 1)
], ['Name', 'Age', 'Gender', 'Salary', 'DeptId'])
departments = spark.createDataFrame([
    (1, 'HR'),
    (2, 'Marketing')
], ['DeptId', 'DeptName'])
projects = spark.createDataFrame([
    (1, 'Project A'),
    (2, 'Project B')
], ['DeptId', 'ProjectName'])
employees.createOrReplaceTempView('employees')
departments.createOrReplaceTempView('departments')
projects.createOrReplaceTempView('projects')

# Complex SQL query involving multiple joins and subqueries
spark.sql('''
SELECT e.Name, e.Age, d.DeptName, p.ProjectName
FROM employees e
JOIN departments d ON e.DeptId = d.DeptId
JOIN projects p ON e.DeptId = p.DeptId
''').show()


24/10/24 11:14:26 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----+---+---------+-----------+
| Name|Age| DeptName|ProjectName|
+-----+---+---------+-----------+
|  Lee| 23|       HR|  Project A|
|James| 34|       HR|  Project A|
| Anna| 28|Marketing|  Project B|
+-----+---+---------+-----------+

